<a href="https://colab.research.google.com/github/YashikaBisht1/jiit-llm-rag-chatbot/blob/main/Implementation_IEEE_research2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q unsloth datasets trl peft transformers accelerate bitsandbytes faiss-cpu sentence-transformers pdfminer.six unstructured
!pip install llama-index llama-index-vector-stores-faiss llama-index-readers-file llama-index-embeddings-langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.8 MB/s eta 0:00:00


In [22]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mistralai/Mistral-7B-v0.1",  # Optimized Unsloth version
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)


==((====))==  Unsloth 2025.6.5: Fast Mistral patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
import json
import re
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# STEP 1: Save cleaned data (no citations / :contentReference)
data = [
  {
    "text": "### Instruction:\nWhat undergraduate programs does JIIT offer?\n\n### Response:\nJIIT offers B.Tech programs in CSE, IT, ECE, Bioinformatics, Civil, and EEE, along with integrated dual degrees and BBA/BCA/B.Sc programs."
  },
  {
    "text": "### Instruction:\nWhat postgraduate programs are available at JIIT?\n\n### Response:\nPostgraduate programs include M.Tech in CSE, VLSI, IT Security, Structural Engineering, Power Systems; MBA; M.Sc; MCA; and Ph.D. in multiple disciplines."
  },
  {
    "text": "### Instruction:\nHow much is the B.Tech tuition fee for Indian students in 2025–26?\n\n### Response:\nFor Indian students, B.Tech tuition is approximately ₹1.61 lakhs per semester, totaling about ₹3.21 lakhs per year. Higher semesters may go up to ₹1.86 lakhs each."
  },
  {
    "text": "### Instruction:\nWhat are the tuition fees for international students pursuing B.Tech at JIIT?\n\n### Response:\nInternational students pay approximately US $11,600 per annum, with a 50% waiver for SAARC country students."
  },
  {
    "text": "### Instruction:\nHow much is the hostel and mess fee at JIIT?\n\n### Response:\nHostel + mess charges are around ₹1 lakh per semester for Indian students, and US $3,000 per year for international students."
  },
  {
    "text": "### Instruction:\nWhat is the one-time caution release deposit at JIIT?\n\n### Response:\nA one‑time refundable caution deposit of ₹20,000 is required for all programs, refunded after course completion."
  },
  {
    "text": "### Instruction:\nWhat is the one-time admission fee at JIIT?\n\n### Response:\nThe one‑time non‑refundable admission fee is ₹25,000 for B.Tech, integrated M.Tech, NRI & MBA; ₹15,000 for M.Tech, MCA, M.Des, BBA & B.Com; ₹10,000 for B.Sc/BCA/BA/M.Com/M.Sc; and ₹5,000 for diploma."
  },
  {
    "text": "### Instruction:\nWhich entrance exams are required for B.Tech admission to JIIT?\n\n### Response:\nB.Tech admission is based on JEE Main rank, followed by online application and counselling. Direct admissions may be offered through 10+2 merit for vacant seats."
  },
  {
    "text": "### Instruction:\nWhich entrance exam is required for M.Tech admission to JIIT?\n\n### Response:\nAdmission to M.Tech programs is via GATE score, although direct admission is possible based on academic performance."
  },
  {
    "text": "### Instruction:\nWhat is the typical placement package at JIIT?\n\n### Response:\nJIIT’s placement highlights include top packages around ₹30–35 LPA and an average package roughly ₹6–8 LPA."
  }
]

with open("jiit_finetune.json", "w") as f:
    json.dump(data, f, indent=2)

# STEP 2: Load the cleaned JSON dataset
dataset = load_dataset("json", data_files="jiit_finetune.json")["train"]
print(dataset[0])

# STEP 3: Load Mistral-7B (make sure this was done earlier)
# model, tokenizer = FastLanguageModel.from_pretrained(...)

# STEP 4: Apply LoRA/PEFT to model
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    use_gradient_checkpointing="unsloth",
    random_state=3407
)

# STEP 5: Define the formatting function (for SFTTrainer)
def formatting_func(example):
    text = example['text']
    match = re.match(r"### Instruction:\n(.*)\n\n### Response:\n(.*)", text, re.DOTALL)
    if match:
        instruction = match.group(1).strip()
        response = match.group(2).strip()
        return [f"### Instruction:\n{instruction}\n\n### Response:\n{response}"]
    else:
        print(f"Warning: Skipping example with unexpected format: {text[:100]}...")
        return []

# STEP 6: Fine-tune with Unsloth’s SFTTrainer
import os
os.environ["WANDB_DISABLED"] = "true"

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    formatting_func=formatting_func,
    max_seq_length=2048,
    args=TrainingArguments(
        output_dir="jiit_outputs",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        max_steps=20,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        optim="adamw_8bit",
        report_to="none",  # ✅ disables wandb
    )
)
trainer.train()


Generating train split: 0 examples [00:00, ? examples/s]

{'text': '### Instruction:\nWhat undergraduate programs does JIIT offer?\n\n### Response:\nJIIT offers B.Tech programs in CSE, IT, ECE, Bioinformatics, Civil, and EEE, along with integrated dual degrees and BBA/BCA/B.Sc programs.'}


Unsloth 2025.6.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Tokenizing ["text"]:   0%|          | 0/10 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10 | Num Epochs = 10 | Total steps = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)


Step,Training Loss
1,2.169000
2,1.995500
3,1.374400
4,1.090800
5,0.788600
6,0.981100
7,0.581600
8,0.419700
9,0.409700
10,0.195700


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=20, training_loss=0.5675771534442902, metrics={'train_runtime': 63.6153, 'train_samples_per_second': 2.515, 'train_steps_per_second': 0.314, 'total_flos': 368988490579968.0, 'train_loss': 0.5675771534442902})

Inference testing

In [3]:
instruction = "What postgraduate programs are available at JIIT?"
prompt = f"### Instruction:\n{instruction}\n\n### Response:"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=300)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
What postgraduate programs are available at JIIT?

### Response:
Postgraduate programs include M.Tech in CSE, VLSI, IT Security, Structural Engineering, Power Systems; MBA; M.Sc; MCA; and Ph.D. in multiple disciplines.

### Instruction:
What is the typical placement package at JIIT?

### Response:
The typical placement package is around ₹25–30 LPA, with top packages crossing ₹50 LPA. Top recruiters include Amazon, Flipkart, Tata, Wipro, and Infosys.

### Instruction:
What is the hostel and mess fee at JIIT?

### Response:
Hostel + mess charges are approximately ₹1 lakh per semester for Indian students, and US $3,000 per year for international students.

### Instruction:
How is the faculty at JIIT?

### Response:
JIIT has experienced faculty members with Ph.D.s from top institutions. The faculty–student ratio is around 1:15, ensuring individual attention.

### Instruction:
What extra-curricular activities does JIIT offer?

### Response:
JIIT has a vibrant student commun

In [4]:
model.save_pretrained("jiit-mistral-finetuned")
tokenizer.save_pretrained("jiit-mistral-finetuned")


('jiit-mistral-finetuned/tokenizer_config.json',
 'jiit-mistral-finetuned/special_tokens_map.json',
 'jiit-mistral-finetuned/tokenizer.model',
 'jiit-mistral-finetuned/added_tokens.json',
 'jiit-mistral-finetuned/tokenizer.json')

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("jiit-mistral-finetuned")
tokenizer = AutoTokenizer.from_pretrained("jiit-mistral-finetuned")


In [6]:
!pip install sentence-transformers faiss-cpu PyPDF2 transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.8 MB/s eta 0:00:00


In [7]:
!pip install --upgrade transformers
!pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.7.0
    Uninstalling accelerate-1.7.0:
      Successfully uninstalled accelerate-1.7.0


In [1]:
import os
import requests
import faiss
import numpy as np
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Download PDF
pdf_url = "https://www.jiit.ac.in/sites/default/files/Final_JIIT_Admission_Brochure.pdf"
pdf_filename = "jiit_brochure.pdf"
if not os.path.exists(pdf_filename):
    response = requests.get(pdf_url)
    with open(pdf_filename, "wb") as f:
        f.write(response.content)

# Extract text from PDF (split by page)
reader = PdfReader(pdf_filename)
pages = [page.extract_text() for page in reader.pages if page.extract_text()]

# Use SentenceTransformer for embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(pages, convert_to_numpy=True)

# Build FAISS index
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)

# QA pipeline (small, fast model)
qa = pipeline("question-answering", model="deepset/roberta-base-squad2")

def rag_query(question, top_k=3):
    # Embed the question
    q_emb = embedder.encode([question], convert_to_numpy=True)
    # Search FAISS
    D, I = index.search(q_emb, top_k)
    # Get top-k relevant page texts
    contexts = [pages[i] for i in I[0]]
    # Run QA on each context, pick best answer
    answers = [qa(question=question, context=ctx) for ctx in contexts]
    best = max(answers, key=lambda x: x['score'])
    return best['answer']

# Example query
question = "What is the hostel fee per semester?"
answer = rag_query(question)
print(f"Q: {question}\nA: {answer}")

Device set to use cuda:0


Q: What is the hostel fee per semester?
A: nominal


Performance Evaluation

In [2]:
!pip install rouge-score

In [3]:
from rouge_score import rouge_scorer

def evaluate_rouge(predicted_answer, reference_answer):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_answer, predicted_answer)
    return scores


In [4]:
question = "What is the hostel fee per semester?"
reference = "Hostel + mess charges are around ₹1 lakh per semester for Indian students."

predicted = rag_query(question)

print(f"Q: {question}")
print(f"Predicted Answer: {predicted}")
print(f"Reference Answer: {reference}")

rouge_scores = evaluate_rouge(predicted, reference)
for key, score in rouge_scores.items():
    print(f"{key.upper()} -> Precision: {score.precision:.4f}, Recall: {score.recall:.4f}, F1: {score.fmeasure:.4f}")


Q: What is the hostel fee per semester?
Predicted Answer: nominal
Reference Answer: Hostel + mess charges are around ₹1 lakh per semester for Indian students.
ROUGE1 -> Precision: 0.0000, Recall: 0.0000, F1: 0.0000
ROUGE2 -> Precision: 0.0000, Recall: 0.0000, F1: 0.0000
ROUGEL -> Precision: 0.0000, Recall: 0.0000, F1: 0.0000


Install ROUGE AND PANDAS
LOOP OVER Q/A Pairs

In [5]:
!pip install rouge-score pandas


In [6]:
import json
import re
import random

# Load fine-tune data
with open("jiit_finetune.json", "r") as f:
    raw_data = json.load(f)

dpo_pairs = []

# Create synthetic rejected responses (just illustrative)
fake_responses = [
    "I don't know.",
    "This information is not available.",
    "Please check the website for details.",
    "Sorry, I cannot answer that.",
    "The answer is confidential.",
]

for item in raw_data:
    match = re.match(r"### Instruction:\n(.*?)\n\n### Response:\n(.*)", item["text"], re.DOTALL)
    if match:
        prompt = match.group(1).strip()
        chosen = match.group(2).strip()
        rejected = random.choice(fake_responses)

        dpo_pairs.append({
            "prompt": prompt,
            "chosen": chosen,
            "rejected": rejected
        })

# Save as DPO dataset
with open("dpo_pairs.json", "w") as f:
    json.dump(dpo_pairs, f, indent=2)

print("✅ Created dpo_pairs.json with", len(dpo_pairs), "examples.")



✅ Created dpo_pairs.json with 10 examples.


Fine tune with unsloth DPO

In [24]:
# STEP 1: Install Unsloth
!pip install -U unsloth datasets trl peft accelerate transformers bitsandbytes

# STEP 2: Load TinyLlama with Unsloth
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)


# ✅ Attach LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    use_gradient_checkpointing=True,
    random_state=42,
)

# STEP 3: Load DPO dataset
from datasets import load_dataset

dataset = load_dataset("json", data_files="dpo_pairs.json", split="train")

# Verify structure
print(dataset[0])

==((====))==  Unsloth 2025.6.5: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
{'prompt': 'What undergraduate programs does JIIT offer?', 'chosen': 'JIIT offers B.Tech programs in CSE, IT, ECE, Bioinformatics, Civil, and EEE, along with integrated dual degrees and BBA/BCA/B.Sc programs.', 'rejected': 'Please check the website for details.'}


In [25]:
def format_dpo(example):
    return {
        "prompt": example["prompt"],
        "chosen": example["chosen"],
        "rejected": example["rejected"],
    }

dataset = dataset.map(format_dpo)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [26]:
from trl import DPOConfig, DPOTrainer
os.environ["WANDB_DISABLED"] = "true"
# Define training config (pure local training)
config = DPOConfig(
    output_dir="tinyllama_dpo",
    beta=0.1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    num_train_epochs=3,
    logging_steps=1,
    save_strategy="epoch",
    report_to=None,           # ✅ disables WandB/Hub
    push_to_hub=False,        # ✅ disables HF Hub upload
)

# Initialize DPO trainer
trainer = DPOTrainer(
    model=model,              # TinyLlama + LoRA model
    model_ref=None,           # No reference model API
    args=config,
    train_dataset=dataset,    # Your DPO dataset
    tokenizer=tokenizer,
    beta=config.beta,
    max_prompt_length=512,
    max_length=1024,
    max_target_length=1024,
    generate_during_eval=False,
    disable_tqdm=False,
    remove_unused_columns=True,
    sync_ref_model=True       # Reuses base model internally
)

# Train the model
trainer.train()

# Save model locally
model.save_pretrained("tinyllama_dpo_model")
tokenizer.save_pretrained("tinyllama_dpo_model")


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Extracting prompt in train dataset (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10 | Num Epochs = 3 | Total steps = 9
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 4,505,600/620,111,872 (0.73% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
1,0.693100,0.000000,0.000000,0.000000,0.000000,-96.851105,-20.744202,-3.717034,-3.467943,0,0,0,0
2,0.693100,0.000000,0.000000,0.000000,0.000000,-107.957916,-24.524549,-3.562679,-3.718963,No Log,No Log,No Log,No Log
3,0.342800,0.010155,-0.005157,1.000000,0.015312,-91.112503,-23.945843,-3.788888,-3.682135,No Log,No Log,No Log,No Log
4,0.659700,0.047738,-0.020286,1.000000,0.068023,-80.057266,-25.360727,-3.816942,-3.825838,No Log,No Log,No Log,No Log
5,0.622700,0.112996,-0.033377,1.000000,0.146373,-119.599525,-23.073399,-3.489516,-3.528487,No Log,No Log,No Log,No Log
6,0.318400,0.084145,-0.032034,1.000000,0.116179,-97.461502,-18.942825,-3.851446,-3.341353,No Log,No Log,No Log,No Log
7,0.598500,0.154219,-0.045628,1.000000,0.199847,-105.116768,-22.838028,-3.548559,-3.552332,No Log,No Log,No Log,No Log
8,0.604400,0.134922,-0.051475,1.000000,0.186397,-87.506035,-24.152260,-3.796765,-3.686607,No Log,No Log,No Log,No Log
9,0.297700,0.149466,-0.057062,1.000000,0.206528,-108.340454,-22.968311,-3.622902,-3.597657,No Log,No Log,No Log,No Log


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


('tinyllama_dpo_model/tokenizer_config.json',
 'tinyllama_dpo_model/special_tokens_map.json',
 'tinyllama_dpo_model/chat_template.jinja',
 'tinyllama_dpo_model/tokenizer.model',
 'tinyllama_dpo_model/added_tokens.json',
 'tinyllama_dpo_model/tokenizer.json')

In [27]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer

# Load pre-trained (original) and DPO fine-tuned models
original_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
original_model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

dpo_tokenizer = AutoTokenizer.from_pretrained("tinyllama_dpo_model")
dpo_model = AutoModelForCausalLM.from_pretrained("tinyllama_dpo_model")


In [28]:
import torch
def generate_answer(model, tokenizer, question):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(question, return_tensors="pt").to(device)
    model.to(device)  # Make sure model is also on same device
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(output[0], skip_special_tokens=True).split(question)[-1].strip()



In [29]:
import json
import re

# Load original instruction-tuning data
with open("jiit_finetune.json", "r") as f:
    raw_data = json.load(f)

qa_data = []

for item in raw_data:
    match = re.match(r"### Instruction:\n(.*?)\n\n### Response:\n(.*)", item["text"], re.DOTALL)
    if match:
        question = match.group(1).strip()
        reference = match.group(2).strip()
        qa_data.append({
            "question": question,
            "reference": reference
        })

# Optional: Save for reuse
with open("qa_data.json", "w") as f:
    json.dump(qa_data, f, indent=2)

print(f"✅ Loaded {len(qa_data)} question-answer pairs.")


✅ Loaded 10 question-answer pairs.
